# Query Expansion

0) Just some imports

In [2]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [3]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeIDFs(self, documents):
        self.idfs = []
        for term in self.terms:
            numOfDocsContainingTerm = 0
            for doc in documents:
                if term in doc.tokens:
                    numOfDocsContainingTerm += 1
            self.idfs.append(numOfDocsContainingTerm)
        print(self.idfs)
        for i in range(len(self.idfs)):
            self.idfs[i] = math.log10(len(documents)/self.idfs[i])
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeCorM(self, documents):
        aT = []
        for i, doc in enumerate(documents):
            aT.append(np.ones(0))
            for num in doc.bow:
                aT[i] = np.append(aT[i], num)
        aT = np.array(aT)
        a = np.transpose(aT)
        for i, row in enumerate(a):
            suma = 0
            for num in row:
                suma += num ** 2
            a[i] /= suma ** 0.5
        aT = np.transpose(a)
        self.corM = np.dot(a, aT)
        for i, row in enumerate(self.corM):
            self.corM[i][i] = -1
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [4]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [5]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [6]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        self.bow = [0] * len(self.dictionary.terms)
        for token in self.tokens:
            self.bow[self.dictionary.terms.index(token)] += 1
    
    ### TODO - complete this method; it should compute a TF representation
    def computeTF_Representation(self):
        self.tf = []
        for num in self.bow:
            self.tf.append(num/len(self.tokens))
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        self.tf_idf = []
        for i in range(len(self.tf)):
            self.tf_idf.append(self.tf[i] * dictionary.idfs[i])
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [7]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[2, 4, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 8, 1, 1, 42, 2, 1, 1, 7, 3, 2, 1, 1, 2, 1, 1, 6, 1, 1, 1, 5, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 5, 1, 1, 1, 1, 4, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 5, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 4, 12, 3, 3, 12, 1, 1, 1, 1, 3, 1, 4, 1, 1, 1, 1, 4, 1, 3, 1, 2, 1, 1, 1, 1, 12, 2, 2, 4, 1, 2, 2, 1, 1, 3, 1, 1, 36, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 3, 1, 1, 4, 1, 2, 1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 26, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 8, 1, 1, 1, 2, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 20, 1, 1, 1, 4, 2, 1, 3, 1, 1, 1, 1, 1, 15, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 4, 3, 2, 1, 14, 1, 1, 5, 2, 1, 1, 2, 2, 9, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 6, 1, 2, 3, 1, 1, 1, 1, 2, 7, 1, 3, 4, 3, 2, 1, 86, 1, 1, 2, 2, 1, 1, 3, 1, 3, 2, 4, 1, 1, 1, 86, 2, 2, 1, 3, 2, 1, 2, 2, 1, 1, 1, 1

1.5) Compute the document representations (for each document run computeRepresentations())

In [8]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [9]:
### TODO 
def getSimilarity(v1, v2):
    numerator = v1length = v2length = 0
    for i in range(len(v1)):
        numerator += v1[i] * v2[i]
        v1length += v1[i] ** 2
        v2length += v2[i] ** 2
    denominator = v1length ** 0.5 * v2length ** 0.5
    return numerator / denominator

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [10]:
#query = "machine learning"
#query = "academic research"
query = "international conference"
#query = "international conference washington"

In [11]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.751942853521094 | DOC ID = 86
International Conferences on Machine Learning and Applications
 The 2002 International Conferences on Machine Learning and Applications The
 2003 International Conferences on Machine Learning and Applications. 


RANK = 2 WITH SIMILARITY = 0.5065902846250935 | DOC ID = 19
1998 International Machine Learning Conference
 (larger version of cover). The Fifteenth International Conference on Machine Learning.
 The on-line schedule for ICML-98 contains links to many of the papers. ... 
 Description: The Fifteenth International Conference on Machine Learning. July 24-27, 1998 in Madison, Wisconsin.


RANK = 3 WITH SIMILARITY = 0.3570916080352588 | DOC ID = 41
ICML-2000
 Seventeenth International Conference on Machine Learning. Stanford University. ... Tutorials
 on Commercial Applications of Machine Learning and Data Mining [new]. ... 
 Description: Seventeenth International Conference on Machine Learning. Stanford University June 29-

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [12]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -1.          0.         ...  0.18898224  0.
   0.        ]
 [ 0.          0.         -1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.18898224  0.         ... -1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -1.        ]]


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [15]:
query = "machine"
#query = "algorithm"
#query = "learning"
#query = "conference"
#query = "research"
#query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    qd = Document(-1, query, dictionary)
    result = []
    for token in qd.tokens:
        result.append(dictionary.terms[np.where(dictionary.corM[dictionary.terms.index(token)] == np.amax(dictionary.corM[dictionary.terms.index(token)]))[0][0]])
    return result
        
suggestKeywords(query, dictionary)

['learning']

# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [31]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    qd.tf_idf = alpha * np.array(qd.tf_idf)
    relevantVector = np.zeros(len(dictionary.terms))
    for ind in rel_docs:
        relevantVector += documents[ind].tf_idf
    notRelevantVector = np.zeros(len(dictionary.terms))
    for ind in nrel_docs:
        notRelevantVector += documents[ind].tf_idf
    qd.tf_idf = qd.tf_idf + beta / len(rel_docs) * relevantVector - gamma / len(nrel_docs) * notRelevantVector
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

RANK = 1 WITH SIMILARITY = 0.6836043456758913 | DOC ID = 29
Machine Learning
 6.858/18.428: Machine Learning. ... This course deals with the following topics:
 Formal models of machine learning; Learning concepts from examples; ... 


RANK = 2 WITH SIMILARITY = 0.5213504178311974 | DOC ID = 36
Machine Learning Group
 [ Bristol CS | Index | Research | ML group | Student projects ] Machine
 Learning Research Group. Overview. The Machine Learning Research ... 
 Description: Research on higher-order concept learning, inductive logic programming, multi-agent learning systems,...


RANK = 3 WITH SIMILARITY = 0.1508543967893362 | DOC ID = 13
UTCS Machine Learning Research Group
 Machine learning is the study of adaptive computational systems that
 improve their performance with experience. The UT Machine Learning ... 
 Description: Research on General Inductive Learning, Inductive Logic Programming, Natural Language Learning, Qualitati...


RANK = 4 WITH SIMILARITY = 0.1287323793292721 | DOC 

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [34]:
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [35]:
wn.synsets('machine')

[Synset('machine.n.01'),
 Synset('machine.n.02'),
 Synset('machine.n.03'),
 Synset('machine.n.04'),
 Synset('machine.n.05'),
 Synset('car.n.01'),
 Synset('machine.v.01'),
 Synset('machine.v.02')]

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [39]:
#TODO
for synset in wn.synsets('machine'):
    print(synset.definition())

any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks
an efficient person
an intricate organization that accomplishes its goals efficiently
a device for overcoming resistance at one point by applying force at some other point
a group that controls the activities of a political party
a motor vehicle with four wheels; usually propelled by an internal combustion engine
turn, shape, mold, or otherwise finish by machinery
make by machinery


2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [41]:
#TODO
for synset in wn.synsets('machine'):
    print(synset.hypernyms())

[Synset('device.n.01')]
[Synset('person.n.01')]
[Synset('organization.n.01')]
[Synset('mechanical_device.n.01')]
[Synset('organization.n.01')]
[Synset('motor_vehicle.n.01')]
[Synset('shape.v.02')]
[Synset('produce.v.02')]


See: http://www.nltk.org/howto/wordnet.html
for more examples

In [62]:
print(wn.lemmas('słuchawki', lang='pol'), wn.lemmas('chrząszcz', lang='pol'))

[Lemma('headset.n.01.słuchawki')] [Lemma('beetle.n.01.chrząszcz')]


In [61]:
moustache = wn.synset('moustache.n.01')
beard = wn.synset('beard.n.01')
print(moustache.path_similarity(beard))

0.3333333333333333
